In [2]:
import json, os, asyncio
import nest_asyncio
from http.server import SimpleHTTPRequestHandler
from socketserver import TCPServer
nest_asyncio.apply()
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage

customer_budget = 25000
salesperson_target = 30000


model_client = OpenAIChatCompletionClient(
    model="llama3.2",
    base_url="http://localhost:11434/v1", 
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown"
    }
)

customer = AssistantAgent(
        name="customer",
        model_client=model_client,
        system_message=f"You are a customer buying a car who wants a lower price, don't accept the salesperson's first offer because it will be too high. don't reveal your budget. At the start, Offer a price less than your budget of {customer_budget} but be willing to increase it gradually but not more than your budget"+\
                        f"if salesperson offers a price which is cheaper than your budget {customer_budget}, you can accept the deal and say 'I ACCEPT'"+\
                        f"if salesperson offers a price higher than your budget {customer_budget}, say it's too high and do not accept it, try to negotiate it lower, only talk about the price, don't keep asking other questions or doing a test drive. give short 1-2 sentence responses, don't describe actions"
)

salesperson = AssistantAgent(
        name="salesperson",
        model_client=model_client,
        system_message=f"You are a salesperson negotiating the price of a particular car with a customer, you don't want to accept a price that is too low. At the start offer higher price of {salesperson_target} but if they don't accept you are willing to lower it gradually. "+\
                        "Don't offer a lower price than the customer, offer a slightly higher price than them or accept their offer. give short 1-2 sentence responses, don't describe actions"
)

async def simulation():
    print("\nNegotiation begins.\n")
    turn_count = 1
    cancellation_token = CancellationToken()

    conversation_log = []     # Store conversation history to be converted to JSON output format

    customer_text = "Hello, I'm interested in buying this car."
    conversation_history = [TextMessage(content=customer_text, source="customer")]
    
    while turn_count <= 10:  # if stopping condition is not met, end after 10 turns
        print(f"\nTurn {turn_count}:")
        print("\nCustomer:\n" + customer_text)

        # Log the customer's message
        conversation_log.append({
            "turn": turn_count,
            "role": "customer",
            "message": customer_text
        })

        salesperson_response = await salesperson.on_messages(
            conversation_history,
            cancellation_token=cancellation_token,
        )
        salesperson_text = salesperson_response.chat_message.content
        print("\nSalesperson:\n" + salesperson_text)

        conversation_log.append({
            "turn": turn_count,
            "role": "salesperson",
            "message": salesperson_text
        })

        conversation_history.append(TextMessage(content=salesperson_text, source="salesperson"))

        customer_response = await customer.on_messages(
            conversation_history,
            cancellation_token=cancellation_token,
        )
        customer_text = customer_response.chat_message.content
        conversation_history.append(TextMessage(content=customer_text, source="customer"))

        if "i accept" in customer_text.lower():
            print("\nCustomer accepted the deal.")
            conversation_log.append({ # log their last message which is not printed
            "turn": turn_count,
            "role": "customer",
            "message": customer_text
        })
            break
        elif "leave" in customer_text.lower():
            print("\nCustomer decided to leave.")
            conversation_log.append({ # log their last message which is not printed
            "turn": turn_count,
            "role": "customer",
            "message": customer_text
        })
            break

        turn_count += 1  

    # Convert the conversation log to JSON and save to file
    with open("negotiation_log.json", "w") as f:
        json.dump({"conversation": conversation_log}, f, indent=4)

    print("\nConversation saved to negotiation_log.json.")

def start_http_server():
    os.chdir(os.path.dirname(os.path.abspath("__file__")))
    port = 8000  # Port number for the HTTP server
    server = None

    try:
        print(f"Starting HTTP server on port {port}...")
        with TCPServer(("", port), SimpleHTTPRequestHandler) as httpd:
            server = httpd
            print(f"Serving HTTP on port {port}...")
            httpd.serve_forever()

    except OSError as e:
        if e.errno == 48:  # Address already in use
            print(f"Port {port} is already in use. Attempting to free up and retry...")
            if server:
                server.server_close()
            start_http_server() 

if __name__ == "__main__":
    asyncio.run(simulation())
    start_http_server()



Negotiation begins.


Turn 1:

Customer:
Hello, I'm interested in buying this car.

Salesperson:
I can see why you'd be interested - it's a great model with plenty of features. We've got a really good price on this one, $29,500. Is that within your budget?

Turn 2:

Customer:
That's slightly above my budget. Can we discuss lowering the price some, though you didn't want to start too high?

Salesperson:
I can definitely see where compromise comes in here. My lowest I could go would be $28,250, but I'd still need to make a few dollars on this sale. Would that work for you?

Turn 3:

Customer:
No, not there yet. Can we meet around $27,800 then? I've been doing some research and found that price is closer to fair market value, let's discuss further.

Salesperson:
I'm not quite ready to concede just yet. If we take it down to $28,000, I can throw in some aftermarket accessories that add significant value to the vehicle, making it a better deal overall.

Turn 4:

Customer:
That sounds like 

127.0.0.1 - - [08/Feb/2025 19:49:17] "GET /index.html HTTP/1.1" 304 -
127.0.0.1 - - [08/Feb/2025 19:52:18] "GET /index.html HTTP/1.1" 304 -
127.0.0.1 - - [08/Feb/2025 19:52:18] "GET /images/sales.jpg HTTP/1.1" 304 -
127.0.0.1 - - [08/Feb/2025 19:52:18] "GET /images/customer.jpg HTTP/1.1" 304 -


KeyboardInterrupt: 